In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Data") \
    .config("spark.driver.extraClassPath", "/opt/homebrew/Cellar/apache-spark/3.5.5/libexec/jars/mysql-connector-j-8.0.33.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/02 14:34:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Reading the cmsoec excel file from Hadoop
df = spark.read.csv("hdfs://localhost:9000/task/Cmosec.xlsx", header = True, inferSchema = True)

In [4]:
#Reading the connecture csv file from Hadoop
df1 = spark.read.csv("hdfs://localhost:9000/task/Connecture.csv", header = True, inferSchema = True)
df1.show(truncate = False)
df1.printSchema()

+----------------+---+---------------+-----------+----+--------------+----------------+
|Name            |ID |Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+----------------+---+---------------+-----------+----+--------------+----------------+
|Adam Hughes     |290|1253240        |PAe42136   |BP2 |2022-01-26    |2023-01-26      |
|Jason Cruz      |545|55086605       |sbc22542   |BP4 |2021-11-11    |2022-11-11      |
|Lori Vega       |287|44328231       |FMQ54961   |BP1 |2021-06-07    |2022-06-07      |
|Amber Gibbs     |863|18368312       |ZoL63944   |BP1 |2022-09-22    |2023-09-22      |
|Mr. Joseph Koch |726|18561095       |kqt06232   |BP1 |2021-04-02    |2022-04-02      |
|James Hill      |976|30788438       |MQe52149   |BP1 |2021-02-11    |2022-02-11      |
|Christine James |773|77641827       |CJF84717   |BP4 |2022-02-11    |2023-02-11      |
|Joan Bowen      |148|36042272       |rSV23123   |BP4 |2022-06-29    |2023-06-29      |
|Brandon Morrison|160|57168422  

In [5]:
#Reading a FLAT File from Local System
df2 = spark.read.text("/Users/hhemantvg/Documents/pyspark/output_files_v2/OSR.txt")

df2.show(truncate=False)
df2.printSchema()

+------------------------------------------------------------------------+
|value                                                                   |
+------------------------------------------------------------------------+
|Name|ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
|Melissa Lawrence|923|14804803|aFP90950|BP3|2021-10-28|2022-10-28        |
|Brianna Meyer|971|66081567|dsD35127|BP1|2021-08-15|2022-08-15           |
|Derrick Castillo|766|17728556|NoG10377|BP1|2020-09-23|2021-09-23        |
|Claudia Mcbride|712|20498733|RPL64361|BP1|2020-10-05|2021-10-05         |
|Claudia Foster|431|71099015|cWE43972|BP3|2022-03-04|2023-03-04          |
|Amanda Franklin|221|33821390|bHK46456|BP1|2022-11-19|2023-11-19         |
|Laura Raymond|131|38259775|vaj28006|BP4|2022-12-05|2023-12-05           |
|Adam Estes|039|68577186|Mkk10748|BP1|2021-09-15|2022-09-15              |
|Kevin Sharp|638|13433946|bOj23728|BP4|2021-11-03|2022-11-03             |
|Cynthia Stewart|702|8746

In [6]:
#Reading the Data from MySQL.
def read_from_mysql_db(table_name):
    df3 = spark.read.format('jdbc').options(
        url='jdbc:mysql://127.0.0.1:3306/Task',
        driver='com.mysql.jdbc.Driver',
        dbtable=table_name,
        user='root',
        password='12345678'
    ).load()
    return df3

df4 = read_from_mysql_db('sunfire')
df4.show()

df4.printSchema()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+-------------------+---+---------------+-----------+----+--------------+----------------+
|               Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+-------------------+---+---------------+-----------+----+--------------+----------------+
|        Eric Watson|180|       33589817|   Cao65780| BP4|    2021-02-18|      2022-02-18|
|       Dustin Craig|065|       25156844|   YUI03927| BP4|    2020-01-11|      2021-01-10|
|         Sara Brown|596|       56808352|   uft20417| BP1|    2022-09-21|      2023-09-21|
|    Shannon Coleman|747|       28473179|   fKz47916| BP3|    2022-01-21|      2023-01-21|
|       Lisa Sanchez|462|       37801359|   hPo69185| BP3|    2022-01-23|      2023-01-23|
|Christopher Sanchez|070|        1913147|   gBF00656| BP1|    2021-03-04|      2022-03-04|
|    Natalie Nichols|733|       36784861|   bqP81319| BP2|    2022-08-25|      2023-08-25|
|   Jessica Jacobson|918|       98906063|   BYD27492| BP3|    2022-04-09|      2023-04-09|

In [7]:
#Performing Repartition for all the Dataframes

connecture_df = df1.repartition(4)
osr_df = df2.repartition(4)
sunfire_df = df4.repartition(4)

In [8]:
#Staging all the data in single DataFrame
load_df = connecture_df.unionByName(df2, allowMissingColumns = True).unionByName(df4, allowMissingColumns = True)
load_df.show(truncate = True)
load_df.drop('value').show()
load_df.count()

+--------------------+---+---------------+-----------+----+--------------+----------------+-----+
|                Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|
+--------------------+---+---------------+-----------+----+--------------+----------------+-----+
|     Gregory Gardner|555|       57637137|   adj83652| BP3|    2021-01-21|      2022-01-21| NULL|
|         Ryan Carter|900|       88285223|   way02722| BP2|    2021-03-27|      2022-03-27| NULL|
|         David Mccoy|135|       65179074|   Khq22280| BP2|    2020-07-06|      2021-07-06| NULL|
|       James Mathews|737|       80670427|   OVO76223| BP2|    2020-11-23|      2021-11-23| NULL|
|         Adam Hardin|880|       26156036|   MDD23179| BP3|    2021-05-09|      2022-05-09| NULL|
|        Joseph Davis|999|         271813|   vhj25683| BP4|    2021-04-23|      2022-04-23| NULL|
|   Michelle Anderson|861|       36294490|   Qvh14280| BP4|    2020-11-06|      2021-11-06| NULL|
|         Lisa Juare

16001

In [9]:
#reading the Newly Generated Fake data to check the Matches with the Loaded Data

def read_mysql(table_name):
    df5 = spark.read.format('jdbc').options(
        url = 'jdbc:mysql://127.0.0.1:3306/Task',
        driver = 'com.mysql.jdbc.Driver',
        dbtable = table_name,
        user = 'root',
        password = '12345678'
    ).load()
    return df5

db_1 = read_mysql('DB')
db_1.show(truncate = False)
db_1.printSchema()

+-----------------+---+---------------+-----------+----+--------------+----------------+
|Name             |ID |Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+-----------------+---+---------------+-----------+----+--------------+----------------+
|Stephanie Perez  |379|45302344       |sMZ70184   |BP1 |2026-08-04    |2027-08-04      |
|Melanie Dickerson|591|16807585       |Azz26715   |BP1 |2024-12-19    |2025-12-19      |
|Rebecca Smith    |968|26371614       |kxB25597   |BP4 |2026-02-07    |2027-02-07      |
|Gabriel Dixon    |558|78731277       |NjK37290   |BP2 |2025-02-18    |2026-02-18      |
|Jessica Schmidt  |124|84302593       |NrV67144   |BP1 |2026-03-08    |2027-03-08      |
|Lee Tapia        |462|6549960        |ZqT44945   |BP1 |2025-03-21    |2026-03-21      |
|William Campbell |186|99340742       |rLT48590   |BP2 |2025-12-02    |2026-12-02      |
|Kathleen Boyd    |618|16172438       |Myb60209   |BP2 |2025-07-26    |2026-07-26      |
|Gregory Stewart  |88

In [10]:
db_1.count()

13250

In [11]:
#Plan-change Joining

Plan_change = load_df.join(db_1, (load_df.Name == db_1.Name) & (load_df.Member_HICN == db_1.Member_HICN) & (load_df.Plan != db_1.Plan))
Plan_change.show()

+-----------------+---+---------------+-----------+----+--------------+----------------+-----+-----------------+---+---------------+-----------+----+--------------+----------------+
|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+-----------------+---+---------------+-----------+----+--------------+----------------+-----+-----------------+---+---------------+-----------+----+--------------+----------------+
|  Daisy Cervantes|883|       44365293|   yNN03492| BP1|    2021-10-26|      2022-10-26| NULL|  Daisy Cervantes|883|       44365293|   yNN03492| BP4|    2024-02-06|      2025-02-05|
|    Daniel Nelson|997|       10215545|   ABL14430| BP4|    2020-11-17|      2021-11-17| NULL|    Daniel Nelson|997|       10215545|   ABL14430| BP2|    2024-10-12|      2025-10-12|
|   Steven Spencer| 84|       40620531|   IcJ54721| BP3|    2021-02-06|      2022-02-06| N

In [12]:
Plan_change.count()

3010

In [13]:
existing_member = load_df.join(db_1, (load_df.Name == db_1.Name) & (load_df.Member_HICN == db_1.Member_HICN) & (load_df.Plan == db_1.Plan))
existing_member.show(truncate = False)

+----------------+---+---------------+-----------+----+--------------+----------------+-----+----------------+---+---------------+-----------+----+--------------+----------------+
|Name            |ID |Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|Name            |ID |Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+----------------+---+---------------+-----------+----+--------------+----------------+-----+----------------+---+---------------+-----------+----+--------------+----------------+
|Misty Grimes    |318|65560118       |fbL00640   |BP3 |2020-02-05    |2021-02-04      |NULL |Misty Grimes    |318|65560118       |fbL00640   |BP3 |2025-03-08    |2026-03-08      |
|Scott Mcdaniel  |493|38363166       |VVr96215   |BP1 |2020-03-06    |2021-03-06      |NULL |Scott Mcdaniel  |493|38363166       |VVr96215   |BP1 |2025-03-04    |2026-03-04      |
|Latasha Carlson |827|41359326       |kVv26495   |BP4 |2022-01-01    |2023-01-01      |NULL |Latasha

In [14]:
Plan_change.count()

3010

In [15]:
new_user_load = load_df.join(db_1,(load_df.Name == db_1.Name) & (load_df.Member_HICN == db_1.Member_HICN),'left_anti')

In [16]:
if 'value' in new_user_load.columns:
    new_user_load = new_user_load.drop('value')

In [17]:
new_user_db = db_1.join(load_df,(db_1.Name == load_df.Name) & (db_1.Member_HICN == load_df.Member_HICN),'left_anti')

In [18]:
if 'value' in new_user_db.columns:
    new_user_db = new_user_db.drop('value')

In [19]:
new_user = new_user_load.unionByName(new_user_db)
new_user.show()

+----------------+---+---------------+-----------+----+--------------+----------------+
|            Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+----------------+---+---------------+-----------+----+--------------+----------------+
|     Eric Guerra|676|       33235935|   puq29329| BP4|    2020-08-14|      2021-08-14|
| Joseph Andersen|981|       27788171|   mvR62082| BP3|    2021-03-22|      2022-03-22|
|     Laura Brown|599|       60289449|   jFB66462| BP1|    2022-02-13|      2023-02-13|
|   Colin Collier|775|       80811226|   CYe40300| BP3|    2021-06-19|      2022-06-19|
|  Hannah Jimenez|894|       49428391|   XPC64464| BP3|    2022-07-31|      2023-07-31|
|      Tina Young|569|        8554090|   bXv99063| BP3|    2022-11-28|      2023-11-28|
|  David Martinez|265|       96344511|   fsq39713| BP2|    2020-08-01|      2021-08-01|
|   James Collins|413|       12080515|   KQf80586| BP2|    2021-06-14|      2022-06-14|
|   Julie Johnson|994|       461

In [27]:
new_user.count()

17965

In [22]:
Plan_change = load_df.join(db_1, (load_df.Name == db_1.Name) & (load_df.Member_HICN == db_1.Member_HICN) & (load_df.Plan != db_1.Plan))
Plan_change.show()

+-----------------+---+---------------+-----------+----+--------------+----------------+-----+-----------------+---+---------------+-----------+----+--------------+----------------+
|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+-----------------+---+---------------+-----------+----+--------------+----------------+-----+-----------------+---+---------------+-----------+----+--------------+----------------+
|  Daisy Cervantes|883|       44365293|   yNN03492| BP1|    2021-10-26|      2022-10-26| NULL|  Daisy Cervantes|883|       44365293|   yNN03492| BP4|    2024-02-06|      2025-02-05|
|    Daniel Nelson|997|       10215545|   ABL14430| BP4|    2020-11-17|      2021-11-17| NULL|    Daniel Nelson|997|       10215545|   ABL14430| BP2|    2024-10-12|      2025-10-12|
|   Steven Spencer| 84|       40620531|   IcJ54721| BP3|    2021-02-06|      2022-02-06| N

In [23]:
Plan_change.count()

3010

In [24]:
#Existing Member with gap

existing_member = load_df.join(db_1, (load_df.Name == db_1.Name) & (load_df.Member_HICN == db_1.Member_HICN) & (load_df.Plan == db_1.Plan) & (load_df.Effective_Date != db_1.Effective_Date))
existing_member.show()

+----------------+---+---------------+-----------+----+--------------+----------------+-----+----------------+---+---------------+-----------+----+--------------+----------------+
|            Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|            Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|
+----------------+---+---------------+-----------+----+--------------+----------------+-----+----------------+---+---------------+-----------+----+--------------+----------------+
|    Misty Grimes|318|       65560118|   fbL00640| BP3|    2020-02-05|      2021-02-04| NULL|    Misty Grimes|318|       65560118|   fbL00640| BP3|    2025-03-08|      2026-03-08|
|  Scott Mcdaniel|493|       38363166|   VVr96215| BP1|    2020-03-06|      2021-03-06| NULL|  Scott Mcdaniel|493|       38363166|   VVr96215| BP1|    2025-03-04|      2026-03-04|
| Latasha Carlson|827|       41359326|   kVv26495| BP4|    2022-01-01|      2023-01-01| NULL| Latash

In [25]:
existing_member.count()

3011

In [43]:
#current year every month member count
from pyspark.sql.functions import year
current_year = load_df.filter(year('Effective_Date') == 2022)
current_year.show()

+-----------------+---+---------------+-----------+----+--------------+----------------+-----+
|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|
+-----------------+---+---------------+-----------+----+--------------+----------------+-----+
|     Shane Molina|230|        1793377|   dLP27005| BP1|    2022-04-20|      2023-04-20| NULL|
|  Jeanette Turner|448|        5261206|   FtP07219| BP3|    2022-11-09|      2023-11-09| NULL|
|Nicholas Ward DDS| 77|       67771536|   JnF59081| BP1|    2022-11-30|      2023-11-30| NULL|
|     Caleb Brooks|805|       73891209|   Yua88288| BP3|    2022-09-05|      2023-09-05| NULL|
|      David Brown|420|       97468660|   qXL32879| BP3|    2022-08-07|      2023-08-07| NULL|
|     Jodi Gilmore|352|       25955293|   wNL19425| BP1|    2022-06-26|      2023-06-26| NULL|
|    Joseph Ingram|861|       83146768|   cSv32798| BP2|    2022-09-02|      2023-09-02| NULL|
| Courtney Hawkins|505|       49254119|   GLJ90224

In [79]:
#Monthwise count
from pyspark.sql.functions import month
month_count = current_year.groupBy(month('Effective_Date').alias('month')).agg(count('*').alias('total_count')).orderBy('month')
month_count.show()

+-----+-----------+
|month|total_count|
+-----+-----------+
|    1|        339|
|    2|        318|
|    3|        320|
|    4|        330|
|    5|        332|
|    6|        343|
|    7|        326|
|    8|        331|
|    9|        343|
|   10|        371|
|   11|        315|
|   12|        347|
+-----+-----------+



In [76]:
monthly_member_count = current_year.groupBy(month('Effective_Date').alias('Month')).agg(count('*').alias('Member_Count')).orderBy('Month')

In [77]:
monthly_member_count.show()

+-----+------------+
|Month|Member_Count|
+-----+------------+
|    1|         339|
|    2|         318|
|    3|         320|
|    4|         330|
|    5|         332|
|    6|         343|
|    7|         326|
|    8|         331|
|    9|         343|
|   10|         371|
|   11|         315|
|   12|         347|
+-----+------------+



In [80]:
#Show only one Month Record
one_month = load_df.filter(month('Effective_Date') == 12)
one_month.show()

+-----------------+---+---------------+-----------+----+--------------+----------------+-----+
|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|
+-----------------+---+---------------+-----------+----+--------------+----------------+-----+
|      Brian Myers|338|       15377793|   ALB67698| BP1|    2022-12-27|      2023-12-27| NULL|
|       Todd Patel|839|       24079700|   lRy41384| BP2|    2020-12-10|      2021-12-10| NULL|
|      Erika Chung|494|       67065954|   ShB90102| BP4|    2021-12-09|      2022-12-09| NULL|
|     Bradley Cruz|112|       81051417|   DAa26924| BP3|    2020-12-21|      2021-12-21| NULL|
|   Sandra Patrick|872|       84707646|   vyz78007| BP2|    2022-12-24|      2023-12-24| NULL|
|       Kurt Foley|983|       37407743|   hpV63803| BP4|    2022-12-16|      2023-12-16| NULL|
|     Glenn Harris|163|       60549308|   hUF09966| BP1|    2022-12-16|      2023-12-16| NULL|
|   Madison Parker|802|       66151455|   Vaf49267

In [101]:
#Generate Row_Number for each Plan for Month 12
from pyspark.sql import Window
from pyspark.sql.functions import row_number

In [102]:
part = Window.partitionBy('Plan').orderBy('Effective_Date')

In [103]:
generate_row = one_month.withColumn('row_No', row_number().over(part))

In [104]:
generate_row.show()

+-----------------+---+---------------+-----------+----+--------------+----------------+-----+------+
|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|row_No|
+-----------------+---+---------------+-----------+----+--------------+----------------+-----+------+
|Steven Stevens II|054|       94805431|   UkQ49898| BP1|    2020-12-01|      2021-12-01| NULL|     1|
|Kathleen Williams|323|        3037421|   htR13756| BP1|    2020-12-02|      2021-12-02| NULL|     2|
|     Lisa Maxwell|600|       69425137|   ELX66214| BP1|    2020-12-02|      2021-12-02| NULL|     3|
|     Linda Miller|636|       17350941|   GPk86244| BP1|    2020-12-02|      2021-12-02| NULL|     4|
|  Jennifer Cooper|991|       17229615|   wjp46299| BP1|    2020-12-03|      2021-12-03| NULL|     5|
|    Jerry Sanders|273|       41648109|   dib80674| BP1|    2020-12-04|      2021-12-04| NULL|     6|
|     Eric Roberts|424|       90096172|   nYh55511| BP1|    2020-12-05|      2021-

In [108]:
spec = Window.partitionBy('Plan').orderBy('Effective_Date')

In [109]:
rows = one_month.withColumn('plan_no', row_number().over(spec))
rows.show()

+-----------------+---+---------------+-----------+----+--------------+----------------+-----+-------+
|             Name| ID|Application_Num|Member_HICN|Plan|Effective_Date|Termination_Date|value|plan_no|
+-----------------+---+---------------+-----------+----+--------------+----------------+-----+-------+
|Steven Stevens II|054|       94805431|   UkQ49898| BP1|    2020-12-01|      2021-12-01| NULL|      1|
|Kathleen Williams|323|        3037421|   htR13756| BP1|    2020-12-02|      2021-12-02| NULL|      2|
|     Lisa Maxwell|600|       69425137|   ELX66214| BP1|    2020-12-02|      2021-12-02| NULL|      3|
|     Linda Miller|636|       17350941|   GPk86244| BP1|    2020-12-02|      2021-12-02| NULL|      4|
|  Jennifer Cooper|991|       17229615|   wjp46299| BP1|    2020-12-03|      2021-12-03| NULL|      5|
|    Jerry Sanders|273|       41648109|   dib80674| BP1|    2020-12-04|      2021-12-04| NULL|      6|
|     Eric Roberts|424|       90096172|   nYh55511| BP1|    2020-12-05|  

In [110]:
rows.count()

1038

In [111]:
r = rows.collect()
for i in r:
    print(i)

Row(Name='Steven Stevens II', ID='054', Application_Num=94805431, Member_HICN='UkQ49898', Plan='BP1', Effective_Date='2020-12-01', Termination_Date='2021-12-01', value=None, plan_no=1)
Row(Name='Kathleen Williams', ID='323', Application_Num=3037421, Member_HICN='htR13756', Plan='BP1', Effective_Date='2020-12-02', Termination_Date='2021-12-02', value=None, plan_no=2)
Row(Name='Lisa Maxwell', ID='600', Application_Num=69425137, Member_HICN='ELX66214', Plan='BP1', Effective_Date='2020-12-02', Termination_Date='2021-12-02', value=None, plan_no=3)
Row(Name='Linda Miller', ID='636', Application_Num=17350941, Member_HICN='GPk86244', Plan='BP1', Effective_Date='2020-12-02', Termination_Date='2021-12-02', value=None, plan_no=4)
Row(Name='Jennifer Cooper', ID='991', Application_Num=17229615, Member_HICN='wjp46299', Plan='BP1', Effective_Date='2020-12-03', Termination_Date='2021-12-03', value=None, plan_no=5)
Row(Name='Jerry Sanders', ID='273', Application_Num=41648109, Member_HICN='dib80674', Pl